In [1]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from typing import Any, Tuple
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
import csv 
import joblib
from dateutil import parser

# This function loads the dataset from the given path and returns the features and target variables, as well as the number of men, average duration of activity, 
# and number of senior users. 
# During evaluation set, the is_eval_dataset flag should be set to True
# During the second task, the is_handball_dataset should be set to True
# Load dataset and preprocess features using pandas
def load_data(dataset_path: str, is_eval_dataset=False, is_handball_dataset=False) -> Tuple[Any, Any, Any, int, float, int]:
    # Remove columns which you consider not relevant for calories prediction
    columns_to_drop = ["User_ID"]
    
    avg_duration = 0
    no_males = 0
    no_senior_users = 0

    # Load dataset from CSV file
    dataset = []
    with open(dataset_path, "r", encoding="utf-8") as file:
        reader = csv.DictReader(file)
        for row in reader:
            # First, remove columns that are not relevant for calories prediction
            for col in columns_to_drop:
                row.pop(col)

            # Convert numerical values to int or float
            for key, value in row.items():
                if value.replace('.', '', 1).isdigit():  # Check if it is a number
                    if '.' in value:
                        row[key] = float(value)  # Convert to float if decimal present
                    else:
                        row[key] = int(value)
                
            # Encode gender if available 
            if not(is_handball_dataset):
                
                if row["Gender"] == 'male':
                    no_males += 1
                    row["Gender"] = 0
                else:
                    row["Gender"] = 1
            dataset.append(row)

            avg_duration += row['Duration']

            if row['Age'] >= 75:
                no_senior_users += 1
            
    # Calculate average price and owners
    avg_duration = round(avg_duration / len(dataset), 2)

    
    target = "Calories"
    # Separate features (X) and target (y)
    X = []
    y = []    
    
    for row in dataset:
        features = {} 
        for key in row:
            if key != target:
                features[key] = row[key]
        X.append(features)
        y.append(row[target] if not is_eval_dataset else 0)

    # Return the output tuple        
    return pd.DataFrame(X), pd.DataFrame(y, columns=['Calories']), no_males, avg_duration, no_senior_users

# Load the full dataset
X, y, no_males, avg_duration, no_senior_users = load_data("Dataset/dataset_train.csv")

def write_answer_task0(candidat_no):
    
    with open(f"../Solutii/Candidat_{candidat_no}/output_0.csv", "w") as file:
        file.write("Samples,No. Males,Average Duration,SeniorUsers\n")
        file.write(f"{len(X)},{no_males},{avg_duration},{no_senior_users}\n")

In [3]:
DROP_COLUMNS = [[], ['User_ID'], ['User_ID', 'Body_Temp'], ['User_ID', 'Heart_Rate'], ['User_ID', 'Duration'], ['User_ID', 'Weight'], ['User_ID', 'Height'], ['User_ID', 'Gender'],
                ['User_ID', 'Body_Temp', 'Heart_Rate'], ['User_ID', 'Body_Temp', 'Duration'], ['User_ID', 'Body_Temp', 'Weight'], ['User_ID', 'Body_Temp', 'Height'], ['User_ID', 'Body_Temp', 'Gender'],
                ['User_ID', 'Heart_Rate', 'Duration'], ['User_ID', 'Heart_Rate', 'Weight'], ['User_ID', 'Heart_Rate', 'Height'], ['User_ID', 'Heart_Rate', 'Gender'], 
                ['User_ID', 'Duration', 'Weight'], ['User_ID', 'Duration', 'Height'], ['User_ID', 'Duration', 'Gender'], ['User_ID', 'Weight', 'Height'], ['User_ID', 'Height', 'Gender'],
                ['User_ID', 'Body_Temp', 'Heart_Rate', 'Duration'], ['User_ID', 'Body_Temp', 'Heart_Rate', 'Weight'], ['User_ID', 'Body_Temp', 'Heart_Rate', 'Height'], 
                ['User_ID', 'Body_Temp', 'Heart_Rate', 'Gender'], ['User_ID', 'Body_Temp', 'Duration', 'Weight'], ['User_ID', 'Body_Temp', 'Duration', 'Height'], 
                ['User_ID', 'Body_Temp', 'Duration', 'Gender'], ['User_ID', 'Body_Temp', 'Weight', 'Height'], ['User_ID', 'Body_Temp', 'Weight', 'Gender'], ['User_ID', 'Body_Temp', 'Height', 'Gender'],
                ['User_ID', 'Duration', 'Weight', 'Height'], ['User_ID', 'Duration', 'Weight', 'Gender'], ['User_ID', 'Duration', 'Height', 'Gender']]

NORMALIZATIONS = [None, 'Standard', 'MinMax', 'Robust']

HIDDEN_LAYERS = [(4,), (8,), (10,), (16,), (32,), (64,), (4, 4), (4, 8), (4, 10), (4, 16), (4, 32), (4, 64), (8, 4), (8, 8), (8, 10), (8, 16), (8, 32), (8, 64), (16, 4), (16, 8), (16, 16), (16, 32), (16, 64), (32, 4),
                 (32, 8), (32, 16), (32, 32), (32, 64), (64, 4), (64, 8), (64, 16), (64, 32), (64, 64), (4, 4, 4), (4, 4, 8), (4, 4, 16), (4, 4, 32), (4, 4, 64), (4, 8, 4), (4, 8, 8),
                 (4, 8, 16), (4, 8, 32), (4, 8, 64), (4, 16, 4), (4, 16, 8), (4, 16, 16), (4, 16, 32), (4, 16, 64), (4, 32, 4), (4, 32, 8), (4, 32, 16), (4, 32, 32), (4, 32, 64), 
                (4, 64, 4), (4, 64, 8), (4, 64, 16), (4, 64, 32), (4, 64, 64), (8, 4, 4), (8, 4, 8), (8, 4, 16), (8, 4, 32), (8, 4, 64), (8, 8, 4), (4, 8, 8), (8, 8, 16), (8, 8, 32), (8, 8, 64), (8, 16, 4), (8, 16, 8), 
                (8, 16, 16), (8, 16, 32), (8, 16, 64), (8, 32, 4), (8, 32, 8), (8, 32, 16), (8, 32, 32), (8, 32, 64),  (8, 64, 4), (8, 64, 8), (8, 64, 16), (8, 64, 32), (8, 64, 64), 
                (16, 4, 4), (16, 4, 8), (16, 4, 16), (16, 4, 32), (16, 4, 64), (16, 8, 4), (16, 8, 8), (16, 8, 16), (16, 8, 32), (16, 8, 64), (16, 16, 4), (16, 16, 8), 
                (16, 16, 16), (16, 16, 32), (16, 16, 64), (16, 32, 4), (16, 32, 8), (16, 32, 16), (16, 32, 32), (16, 32, 64),  (16, 64, 4), (16, 64, 8), (16, 64, 16), (16, 64, 32), (16, 64, 64),
                (32, 4, 4), (32, 4, 8), (32, 4, 16), (32, 4, 32), (32, 4, 64), (32, 8, 4), (32, 8, 8), (32, 8, 16), (32, 8, 32), (32, 8, 64), (32, 16, 4), (32, 16, 8), 
                (32, 16, 16), (32, 16, 32), (32, 16, 64), (32, 32, 4), (32, 32, 8), (32, 32, 16), (32, 32, 32), (32, 32, 64),  (32, 64, 4), (32, 64, 8), (32, 64, 16), (32, 64, 32), (32, 64, 64),
                (64, 4, 4), (64, 4, 8), (64, 4, 16), (64, 4, 32), (64, 4, 64), (64, 8, 4), (64, 8, 8), (64, 8, 16), (64, 8, 32), (64, 8, 64), (64, 16, 4), (64, 16, 8), 
                (64, 16, 16), (64, 16, 32), (64, 16, 64), (64, 32, 4), (64, 32, 8), (64, 32, 16), (64, 32, 32), (64, 32, 64),  (64, 64, 4), (64, 64, 8), (64, 64, 16), (64, 64, 32), (64, 64, 64)]

LEARNING_RATES = ['constant', 'invscaling', 'adaptive']

SOLVERS = ['lbfgs', 'sgd', 'adam']

LR_INITS = [1e-2, 5e-3, 1e-3, 5e-4, 1e-4, 5e-5, 1e-5]

MOMENTUMS = [1, 0.99, 0.95, 0.9, 0.85]


In [4]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.neural_network import MLPRegressor


def experiment(candidate_no, dc, normalization, hl, solver, lr_init, momentum ,lr, iterations):
    df = pd.read_csv("Dataset\dataset_train.csv")

    df_filtered = df.drop(columns=dc)

    if "Gender" not in dc:
        df_filtered["Gender"] = df_filtered["Gender"].map({"male": 1, "female": 0})

    # Separăm caracteristicile și variabila țintă
    X = df_filtered.drop(columns=["Calories"])
    y = df_filtered["Calories"]

    # 4. Împărțim datele în train și test
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # 5. Standardizăm datele
    if normalization == 'Standard':
        scaler = StandardScaler()
    elif normalization == 'MinMax':
        scaler = MinMaxScaler()
    elif normalization == 'Robust':
        scaler = RobustScaler()
    if normalization is not None:
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)
    else:
        X_train_scaled = X_train
        X_val_scaled = X_val

    if solver == 'lbfgs':
        model = MLPRegressor(hidden_layer_sizes=hl, max_iter=iterations, random_state=42)
    elif solver == 'adam':
        model = MLPRegressor(hidden_layer_sizes=hl, max_iter=iterations, learning_rate_init=lr_init ,random_state=42)
    elif solver =='sgd':
        model = MLPRegressor(hidden_layer_sizes=hl, learning_rate=lr, max_iter=iterations, learning_rate_init=lr_init , momentum=momentum, random_state=42)

    model.fit(X_train_scaled, y_train)
    # Facem predicții
    y_pred = model.predict(X_val_scaled)
    # Calculăm MAE
    mae = mean_absolute_error(y_val, y_pred)

    df = pd.read_csv("Dataset/task1_dataset_eval.csv")

    df_filtered = df.drop(columns=dc)

    # Convertim 'Gender' în format numeric (Male = 1, Female = 0)
    if "Gender" not in dc:
        df_filtered["Gender"] = df_filtered["Gender"].map({"male": 1, "female": 0})


    X_test = df_filtered 

    if normalization is not None: 
        X_test = scaler.transform(X_test)

    y_test = model.predict(X_test)


    with open(f"../Solutii/Candidat_{candidate_no}/output_1.csv", "w") as file:
        # Write the header
        file.write("Calories\n")
        # Write the predictions
        for pred in y_test:
            file.write(str(pred) + "\n")

    df = pd.read_csv("Dataset/task2_dataset_eval.csv")

    # Standardizăm datele de test
    df_filtered = df.drop(columns=dc)

    # Convertim 'Gender' în format numeric (Male = 1, Female = 0)
    if "Gender" not in dc:
        # df_filtered["Gender"] = df_filtered["Gender"].map({"male": 1, "female": 0})
        if 'User_ID' in dc:
            df_filtered.insert(loc=0, column='Gender', value=1)
        else:
            df_filtered.insert(loc=1, column='Gender', value=1)

    X_test = df_filtered 

    if normalization is not None: 
        X_test = scaler.transform(X_test)

    y_test = model.predict(X_test)

    # Salvăm rezultatele în output_2.csv
    output_2 = pd.DataFrame(y_test, columns=["Calories"])
    output_2.to_csv(f"../Solutii/Candidat_{candidate_no}/output_2.csv", index=False)

    write_answer_task0(candidate_no)

<>:6: SyntaxWarning: invalid escape sequence '\d'
<>:6: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Mano\AppData\Local\Temp\ipykernel_18564\2181620782.py:6: SyntaxWarning: invalid escape sequence '\d'
  df = pd.read_csv("Dataset\dataset_train.csv")


In [ ]:
import os
import json
candidate_no = 0

for dc in DROP_COLUMNS:
    for normalization in NORMALIZATIONS:
        for hl in HIDDEN_LAYERS:
            for iterations in range(250, 2250, 250):
                for solver in SOLVERS:
                    if solver in ['sgd', 'adam']:
                        for lr_init in LR_INITS: 
                            if solver == 'sgd':
                                for momentum in MOMENTUMS:
                                    for lr in LEARNING_RATES:
                                        candidate_no += 1
                                        if candidate_no >= 12162:
                                            if not os.path.isdir(f"../Solutii/Candidat_{candidate_no}/"):
                                                os.mkdir(f"../Solutii/Candidat_{candidate_no}/")

                                            print(candidate_no, dc, normalization, hl, solver, lr_init, momentum ,lr, iterations)
                                            experiment(candidate_no, dc, normalization, hl, solver, lr_init, momentum ,lr, iterations)
                                            experiment_config = {
                                                'candidate_no': candidate_no,
                                                'DROP_COLUMNS': dc,
                                                'NORMALIZATIONS': normalization,
                                                'HIDDEN_LAYERS': hl,
                                                'iterations': iterations,
                                                'solver': solver,
                                                'lr_init': lr_init,
                                                'momentum': momentum if solver == 'sgd' else None,
                                                'lr': lr,
                                            }

                                            # Save the config to a JSON file in the respective folder
                                            with open(os.path.join(f"../Solutii/Candidat_{candidate_no}/", 'config.json'), 'w') as json_file:
                                                json.dump(experiment_config, json_file, indent=4)

                                            # assert False

12162 [] None (8, 10) sgd 5e-05 0.9 adaptive 1000
12163 [] None (8, 10) sgd 5e-05 0.85 constant 1000
12164 [] None (8, 10) sgd 5e-05 0.85 invscaling 1000
12165 [] None (8, 10) sgd 5e-05 0.85 adaptive 1000
12166 [] None (8, 10) sgd 1e-05 1 constant 1000
12167 [] None (8, 10) sgd 1e-05 1 invscaling 1000
12168 [] None (8, 10) sgd 1e-05 1 adaptive 1000
12169 [] None (8, 10) sgd 1e-05 0.99 constant 1000
12170 [] None (8, 10) sgd 1e-05 0.99 invscaling 1000
12171 [] None (8, 10) sgd 1e-05 0.99 adaptive 1000
12172 [] None (8, 10) sgd 1e-05 0.95 constant 1000
12173 [] None (8, 10) sgd 1e-05 0.95 invscaling 1000
12174 [] None (8, 10) sgd 1e-05 0.95 adaptive 1000
12175 [] None (8, 10) sgd 1e-05 0.9 constant 1000
12176 [] None (8, 10) sgd 1e-05 0.9 invscaling 1000
12177 [] None (8, 10) sgd 1e-05 0.9 adaptive 1000
12178 [] None (8, 10) sgd 1e-05 0.85 constant 1000
12179 [] None (8, 10) sgd 1e-05 0.85 invscaling 1000
12180 [] None (8, 10) sgd 1e-05 0.85 adaptive 1000
12181 [] None (8, 10) sgd 0.01 1

c:\Users\Mano\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(


15122 [] None (16, 4) sgd 0.01 1 invscaling 250


c:\Users\Mano\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(


15123 [] None (16, 4) sgd 0.01 1 adaptive 250


...

37905 [] None (4, 16, 16) sgd 1e-05 0.85 adaptive 250


c:\Users\Mano\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(


37906 [] None (4, 16, 16) sgd 0.01 1 constant 500
37907 [] None (4, 16, 16) sgd 0.01 1 invscaling 500
37908 [] None (4, 16, 16) sgd 0.01 1 adaptive 500
37909 [] None (4, 16, 16) sgd 0.01 0.99 constant 500
37910 [] None (4, 16, 16) sgd 0.01 0.99 invscaling 500
37911 [] None (4, 16, 16) sgd 0.01 0.99 adaptive 500
37912 [] None (4, 16, 16) sgd 0.01 0.95 constant 500
37913 [] None (4, 16, 16) sgd 0.01 0.95 invscaling 500
37914 [] None (4, 16, 16) sgd 0.01 0.95 adaptive 500
37915 [] None (4, 16, 16) sgd 0.01 0.9 constant 500
37916 [] None (4, 16, 16) sgd 0.01 0.9 invscaling 500
37917 [] None (4, 16, 16) sgd 0.01 0.9 adaptive 500
37918 [] None (4, 16, 16) sgd 0.01 0.85 constant 500
37919 [] None (4, 16, 16) sgd 0.01 0.85 invscaling 500
37920 [] None (4, 16, 16) sgd 0.01 0.85 adaptive 500
37921 [] None (4, 16, 16) sgd 0.005 1 constant 500
37922 [] None (4, 16, 16) sgd 0.005 1 invscaling 500
37923 [] None (4, 16, 16) sgd 0.005 1 adaptive 500
37924 [] None (4, 16, 16) sgd 0.005 0.99 constant 50

c:\Users\Mano\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


38430 [] None (4, 16, 16) sgd 1e-05 0.85 adaptive 1500
38431 [] None (4, 16, 16) sgd 0.01 1 constant 1750
38432 [] None (4, 16, 16) sgd 0.01 1 invscaling 1750
38433 [] None (4, 16, 16) sgd 0.01 1 adaptive 1750
38434 [] None (4, 16, 16) sgd 0.01 0.99 constant 1750
38435 [] None (4, 16, 16) sgd 0.01 0.99 invscaling 1750
38436 [] None (4, 16, 16) sgd 0.01 0.99 adaptive 1750
38437 [] None (4, 16, 16) sgd 0.01 0.95 constant 1750
38438 [] None (4, 16, 16) sgd 0.01 0.95 invscaling 1750
38439 [] None (4, 16, 16) sgd 0.01 0.95 adaptive 1750
38440 [] None (4, 16, 16) sgd 0.01 0.9 constant 1750
38441 [] None (4, 16, 16) sgd 0.01 0.9 invscaling 1750
38442 [] None (4, 16, 16) sgd 0.01 0.9 adaptive 1750
38443 [] None (4, 16, 16) sgd 0.01 0.85 constant 1750
38444 [] None (4, 16, 16) sgd 0.01 0.85 invscaling 1750
38445 [] None (4, 16, 16) sgd 0.01 0.85 adaptive 1750
38446 [] None (4, 16, 16) sgd 0.005 1 constant 1750
38447 [] None (4, 16, 16) sgd 0.005 1 invscaling 1750
38448 [] None (4, 16, 16) sgd 0